In [ ]:
%pip install lancedb
%pip install pandas
%pip install pyarrow
%pip install ipywidgets
%pip install jupyter
%pip install sentence-transformers


In [13]:
import lancedb
import pandas as pd
import pyarrow as pa
import numpy as np

uri = "runtime_assets/vecdb"
db = lancedb.connect(uri)

In [ ]:
db.table_names()

In [ ]:
tbl = db.open_table("somedb")
tbl.count_rows()

In [ ]:
tbl.search(None).select(["filename"]).to_pandas().values

In [ ]:
tbl.search(None).select(["content"]).to_pandas().values

In [29]:
# generate an embedding request to ollama  on the sentance 
sentence = "where can I find data about allergy reactions of patients"

import requests
import json

def get_embedding(text):
    response = requests.post(
        "http://localhost:11434/api/embeddings",
        json={"model": "nomic-embed-text", "prompt": text}
    )
    return response.json()["embedding"]

question_embedding = get_embedding(sentence)
# normalize the embedding to unit length
question_embedding = question_embedding / np.linalg.norm(question_embedding)


In [18]:
from sentence_transformers import SentenceTransformer
# generate an embedding request to ollama  on the sentance 
sentence = "which tables show data about blood tests"
# 1. Load a pretrained Sentence Transformer model
model = SentenceTransformer("all-MiniLM-L12-v2")
# 2. Generate embeddings for a list of sentences
sentence_embeddings = model.encode(sentence)
# 3. Normalize the embeddings to unit length
sentence_embeddings = sentence_embeddings / np.linalg.norm(sentence_embeddings)


In [ ]:
df = tbl.search(sentence_embeddings, query_type="vector") \
    .select(["content"]) \
    .limit(10).to_pandas()
df.sort_values(by="content").values


In [ ]:
df.sort_values(by="content").values

In [21]:
rel_tables = "\n".join(df['content'].values)

In [ ]:
prompt = f"""
You are an AI assistant that answers questions about database schemas and tables.
Here are the relevant tables and columns:
{rel_tables}

Please answer the following question. First explain the purpose of the tables relevant to the question, and if needed provide a query to retrieve the data.
Question:{sentence}
"""

print(prompt)

In [ ]:
import requests
# send request to ollama
response = requests.post(
    "http://localhost:11434/api/generate",
    json={
        "model": "llama3.2",
        "prompt": prompt,
        "options": {
            "temperature": 0.1,
        },
        "stream": False
    }
)

# extract the SQL query from response
sql_query = response.json()["response"]
print("Generated SQL query:")
print(sql_query)